We need custom Reducers which append list made out of classes , based on values of attributes in this classes


1 We need to change architecrure of flow. HyDe genareation must be implemented as subgraph
It will share with main hyde scores
But what if we want to keep and check quality of hyde documents(DOne)


In [1]:
# By chat model we mean LLM model which operates with chats 
import os 
import time
import re

import json
import operator
from openai import OpenAI

import threading
from urllib.parse import urlparse

from langchain_openai import ChatOpenAI 
from dotenv import load_dotenv
from langchain_core.prompts import PromptTemplate

from pydantic import BaseModel, Field
from typing import Literal , Optional

## check performance in agent workflow 
from typing import TypedDict, Annotated, List, Dict, Optional, Set
from langchain_core.messages import BaseMessage, AnyMessage, ToolMessage,HumanMessage, AIMessage, SystemMessage
from langgraph.graph import add_messages , START, END , StateGraph
from IPython.display import Image, display 
from langgraph.checkpoint.memory import MemorySaver

from googleapiclient.discovery import build
from urllib.parse import urlparse

# prepare model for embeddings  
from langchain_openai import OpenAIEmbeddings 
from langchain_core.tools import tool, StructuredTool


from sklearn.metrics.pairwise import cosine_similarity
import numpy as np 
import numpy.typing as npt

from tavily import TavilyClient 

import subprocess
from IPython.display import Image, display


d:\MICB_Projects\8_aml_detective\.venv\lib\site-packages\google\api_core\_python_version_support.py:266: FutureWarning: You are using a Python version (3.10.11) which Google will stop supporting in new releases of google.api_core once it reaches its end of life (2026-10-04). Please upgrade to the latest Python version, or at least Python 3.11, to continue receiving updates for google.api_core past that date.
  warnings.warn(message, FutureWarning)


In [2]:
class APIVault():
    def __init__(self):  # Fixed: single underscore
        self._keys = {}

    def add_key(self, key_name: str, key_value: str):
        """Add API key to the vault"""
        if not key_name:
            print("Key name is empty")
        elif not key_value:  # Added check for value
            print(f"Key value is empty for '{key_name}'")
        else:   
            self._keys[key_name] = key_value
        
    def get_key(self, key_name: str):
        """Get an API key from the vault"""
        if key_name not in self._keys:
            print("Key is not present in Vault")
            return None  # Added return
        else:
            return self._keys[key_name]  # Fixed: use _keys dict


In [3]:
## populate classes
key_vault = APIVault()
key_vault.add_key( "openai_key" , os.environ.get("OPENAI_API_KEY")) # personal api key
key_vault.add_key("tavily_key" ,os.environ.get("TAVILY_API_KEY")) # for tavily search and extract
key_vault.add_key("google_key" ,os.environ.get("GOOGLE_API_KEY")) # gor google search engine
key_vault.add_key("openai_key_azure_41" , os.environ.get("OPENAI_API_KEY_AZURE_41")) # for azure project for gpt 4.1
key_vault.add_key("dummy" , os.environ.get("dummy")) # to test error mesage
key_vault.add_key("perplexity_key" , os.environ.get("PERPLEXITY_API_KEY")) # to test error mesage


Key value is empty for 'dummy'


In [4]:
# Support for Google API 
# define search engine
SEARCH_ENGINE_ID = '26dda816634bd4044'

## Azure service 
endpoint = 'https://datam-mhtcc5x5-westeurope.cognitiveservices.azure.com/openai/v1'
# The SDK automatically appends /chat/completions and other paths

# azure https://datam-mhtcc5x5-westeurope.cognitiveservices.azure.com/openai/deployments/gpt-4.1/chat/completions?api-version=2025-01-01-preview
# This is the direct REST API endpoint, Used for raw HTTP requests (curl, requests library, etc.)

## Path to prompts
CURRENT_DIR = os.getcwd()
PROMPTS_TOOLS = os.path.join(CURRENT_DIR , "prompts\prompts_search_tools.txt")
PROMPTS_STR_OUTPUT = os.path.join(CURRENT_DIR , "prompts\prompts_structured_output.txt")

print(PROMPTS_TOOLS)
print(PROMPTS_STR_OUTPUT)


d:\MICB_Projects\8_aml_detective\prompts\prompts_search_tools.txt
d:\MICB_Projects\8_aml_detective\prompts\prompts_structured_output.txt


#### Test OpenAI SDK - Azure Foundry

#### LLM models api definition

In [5]:
# LLM to select tool, populate class with restricted output, [ tools_name_1, ... ,  exit scenario  ]
# short deterministic outputs
llm_tool_selection = ChatOpenAI(
    model="gpt-4.1-mini",  
    temperature=0.2,     
    max_tokens=50, # Max number of tokens to generate. |
    top_p=0.95,          
    timeout=30,  
    max_retries = 2,
    api_key=key_vault.get_key("openai_key")       
) # OpenAI API key. If not passed in will be read from env var OPENAI_API_KEY. |


# Model to create payload for search tools, its pivotal to create exactly 5 queries, add retry logic inside node
llm_search_tool_payload = ChatOpenAI(
    model="gpt-4.1-mini",  
    temperature=0.2,     
    max_tokens=500, # 5 outputs per 5 languages     
    top_p=0.95,          
    timeout=30,  
    max_retries = 2,
    api_key=key_vault.get_key("openai_key")             
)

## we also need to generate names variations in different languages
## output become input to geenerate regular expression , google search allows search modifiers but other search tools may not
llm_names_variation = ChatOpenAI(
    model="gpt-4.1-mini",   # note the hyphen
    temperature=0.0,        # pure extraction/translation
    max_tokens=50,          # we only need one short line
    timeout=30,  
    max_retries = 2,
    api_key=key_vault.get_key("openai_key")    
)



# Translation and key terms extraction from short input
llm_translation_or_terms = ChatOpenAI(
    model="gpt-4.1-mini",  
    temperature=0.1,      
    max_tokens=500,       
    timeout=30,           
    max_retries=2,         
    api_key=key_vault.get_key("openai_key")                         
)


# # generate persona
llm_expert_generation = ChatOpenAI(
    model="gpt-4.1-mini",     # BETTER: Cheaper, perfectly capable for structured output
    temperature=0.2,         # BEST: Near-deterministic for consistent structure
    max_tokens=1000,         
    timeout=60,             
    max_retries=3,           # ADEQUATE: Structured output usually works first try
    api_key=key_vault.get_key("openai_key")   
)
# LengthFinishReasonError: Could not parse response content as the length limit was reached - CompletionUsage(completion_tokens=200, prompt_tokens=426, total_
# APITimeoutError: Request timed out.
# 4.0 mini replaced with 4.1 as it offers larger context allowing to simulate more experts


llm_hyde_generation = ChatOpenAI(
    model="gpt-4.1-mini",     # OPTIMAL: Cost-effective for creative writing
    temperature=0.5,         # BEST: Creative variation for different journalist styles
    max_tokens=500,          
    timeout=60,            
    max_retries=3,
    api_key=key_vault.get_key("openai_key")             
)


########################################### AZURE
# Duw to error below , before we get better result, try personal data.

# Error generating reputation summary: Error code: 400 - {'error': {'message': "The response was filtered due to the prompt triggering Azure OpenAI's content management policy. Please modify your 
# prompt and retry. To learn more about our content filtering policies please read our documentation:
# https://learn.microsoft.com/en-us/answers/questions/1297066/i-keep-getting-this-error-exception-in-chat-messag

#Error generating reputation summary: Error code: 429 - {'error': {'code': 'RateLimitReached', 'message': 
#'Your requests to gpt-4.1 for gpt-4.1 in West Europe have exceeded the token rate limit for your current AIServices S0 pricing tier. This request was for ChatCompletions_Create under OpenAI Language Model Instance API. Please retry after 60 seconds. To increase your default rate limit, visit: https://aka.ms/oai/quotaincrease.'}}

# Summary of content extracted from URL with tavily, applieed per each link
llm_url_content_summary = ChatOpenAI(
    api_key=key_vault.get_key("openai_key"),    
    #base_url = f"{endpoint}" , 
    #api_key = key_vault.get_key("openai_key_azure_41"), 
    model="gpt-4.1",  # BETTER CHOICE: Your original "gpt-4o" was budget-killer
                          # URL content analysis is mostly extraction/summarization
    
    temperature=0.2,      # BEST: Factual analysis needs consistency, not creativity
    max_tokens=2000,     
    top_p=0.95,         
    timeout=60,
    max_retries=3           
)


# Applied for extract_evidence_claims, consolidates results from Multiple summaries
llm_agg_summaries = ChatOpenAI(
    #api_key=key_vault.get_key("openai_key"), 
    base_url = f"{endpoint}" , 
    api_key = key_vault.get_key("openai_key_azure_41") , 

    model="gpt-4.1", 
    temperature=0.1,      
    max_tokens=2000,     
    top_p=0.95,          
    timeout=60,          
    max_retries=3        
)


# Model uses multiple summaries to make conclusion about possible risk 
llm_evaluation = ChatOpenAI( # compare to 4.o should have better instrusction following
    #api_key=key_vault.get_key("openai_key"), 
    base_url = f"{endpoint}" , 
    api_key = key_vault.get_key("openai_key_azure_41") , 
    model="gpt-4.1", # BadRequestError: Error code: 400 - {'error': {'code': 'unknown_model', 'message': 'Unknown model: gpt-4.o', 'details': 'Unknown model: gpt-4.o'}}      
    temperature=0.2,                   
    max_tokens=3000,     
    top_p=0.95, # not applicable for 5 openai family        
    timeout=60,          
    max_retries=3        
)

# BadRequestError: Error code: 400 - {'error': {'message': "Unsupported parameter: 'top_p' is not supported with this model.", 
# 'type': 'invalid_request_error', 'param': 'top_p', 'code': 'unsupported_parameter'}}

## test gpt 5, takes signifficnt time, requires larger timeout
# result = llm_evaluation.invoke("Generate recommendations on how to prepare banana bread")



In [6]:
# Data and Meta for extracted content, per single link

# First model selects tool and return it, then 
# we pass specific System message to LLM with Tools.
# So essentially:
# Get Name of Tool -> Select System Message -> Pass system message to LLM with Tools

ToolsPool = Literal["google_search","perplexity_search","tavily_search","exit_scenario"]

class Tool_Selected(BaseModel):
       search_engine:ToolsPool = Field(default="" , description="Name of tool used to suggest url link") # taken from tool call tool.name

class LinkCollection(BaseModel): # organise for better visual inspection 
       
       displayLink: str = Field(description="The display URL shown in search results (usually domain name)") # passeed from search tool
       link: str = Field(description="The full URL of the search result") # passeed from search tool

       claim_type: str = Field(default="other", description="Each article must fit specific only 1 claim type") # from ContentSummary
       date_published: Optional[str] = Field( default=None, description="Publication date extracted from content (format: YYYY-MM-DD, or 'Unknown' if not found)" ) # from ContentSummary
       search_engine: str = Field(default="" , description="Name of tool used to suggest url link") # taken from tool call tool.name
       severity_level: str = Field(default="", description="Each article must fit specific only 1 severity level") # from ContentSummary
       summary: str = Field(default="", description="Summary of content extracted from URL") # from ContentSummary

       hyde_score: Optional[float] = Field(default=None, description="Max value of similarity between content of web page and HyDe articles") # from similarity score
                                           
       raw_content: str = Field(default="", description="Content extracted from URL") # passed from content parser                                    
                                           
 
 # how to update LinkCollection
dummy_list = [
    LinkCollection(displayLink="example.com", link="https://example.com/article1")
]

for item in dummy_list:
    print(f"{item.displayLink}: hyde_score={item.hyde_score}, summary='{item.summary}'")

# dummy_list[0].dummy_attribute ="engine_1" # ValueError: "LinkCollection" object has no field "dummy_attribute"    
dummy_list[0].search_engine ="engine_1" # so its not immutbale we can modify in place 

print(dummy_list)


example.com: hyde_score=None, summary=''
[LinkCollection(displayLink='example.com', link='https://example.com/article1', claim_type='other', date_published=None, search_engine='engine_1', severity_level='', summary='', hyde_score=None, raw_content='')]


In [12]:
from operator import add 

class UnifiedResearchState(TypedDict):
    search_results_kw_filter: Annotated[List[LinkCollection] , operator.add] # applied during content extraction, filter via regexp


class LinkCollection(BaseModel): # organise for better visual inspection 
       link: str = Field(description="The full URL of the search result") # passeed from search tool
       hyde_score: Optional[float] = Field(default=None, description="Max value of similarity between content of web page and HyDe articles") # from similarity score                          
                             
                                           
 
l1 = [ LinkCollection(link = "url_1", hyde_score=0.1) , LinkCollection(link = "url_2", hyde_score=None), LinkCollection(link = "url_3", hyde_score=0.6) ]
l2 = [ LinkCollection(link = "url_2", hyde_score=0.5) ]


In [16]:
def custom_reducer(existing:list , new:list) -> list: 
    links  = [item.link for item in new]
    scores = [item.hyde_score for item in new]
    url_hyde_map = {link: score for link, score in zip(links, scores)}

    ## now replace values in the original 
    for item in existing: 
        if item.link in url_hyde_map: 
           item.hyde_score =  url_hyde_map.get(item.link)

    return existing


In [17]:
custom_reducer(l1,l2)

[LinkCollection(link='url_1', hyde_score=0.1),
 LinkCollection(link='url_2', hyde_score=0.5),
 LinkCollection(link='url_3', hyde_score=0.6)]